<a href="https://colab.research.google.com/github/nrohrbach/AnalyseAlpineSolaranlagen/blob/main/AnalyseAlpineSolaranlagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geoanalyse alpine Solaranlagen
 Im Rahmen des Projekts "Potenzialanalysen freistehenden Photovoltaikanlagen" sollen in dieser Geoanalyse alle Unterwerke der Netzebene 4 eruiert werden.

In [1]:
pip install geopandas

In [2]:
pip install pandas

In [3]:
pip install fiona

In [4]:
pip install leafmap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.8 MB/s eta 0:00:00


In [5]:
# Libraries installieren
import geopandas as gpd
import pandas as pd
import fiona
import leafmap

## 1. Unterwerke von Swissgrid aus dem Geobasisdatensatz "Elektrische Anlagen mit einer Nennspannung >36kV" filtern

Aus dem täglich aggregierten Datensatz werden alle Stationen gefiltert welche  den Eigentümer "Swissgrid" haben.

In [6]:
# Daten direkt aus data.geo.admin.ch laden
url = "https://data.geo.admin.ch/ch.bfe.elektrische-anlagen_ueber_36/gpkg/2056/ElektrischeAnlagenNennspannungUeber36kV_V1.gpkg"
Station = gpd.read_file(url, layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Station')
StationenSwissgrid = Station.loc[Station.Eigentuemer=="Swissgrid"]

In [7]:
# Die Geobasisdaten haben eine gemischte Geometriespalte. Aus diesem Grund wird die Geometrie gelöscht und anschliessend für alle Unterwerke eine Punktgeometrie aus den Spalten "_x" und "_y" erstellt.
StationenSwissgrid = StationenSwissgrid.drop(columns='geometry')
dfStationenSwissgrid = gpd.GeoDataFrame(StationenSwissgrid, geometry=gpd.points_from_xy(StationenSwissgrid._x,StationenSwissgrid._y))
dfStationenSwissgrid = dfStationenSwissgrid.set_crs('epsg:2056')

In [8]:
# Die Unterwerke werden in einer interaktiven Karte visualisiert.
m = leafmap.Map(height="400px", width="800px")
m.add_basemap("SwissFederalGeoportal.NationalMapColor")
m.add_gdf(dfStationenSwissgrid, layer_name="Stationen")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## 2. Unterwerke aus dem swissTLM3D exportieren
In einem nächsten Schritt werden alle Untwerke aus dem Datensatz "swissTLM3D" von swisstopo geladen. Aufgrund der Datenmenge wurden die nötigen Daten bereits mit FME vorselektioniert und in einem Geopackage gespeichert.

In [ ]:
# Daten werden aus vorselektioniertem Geopackage geladen (Datenstand: 18.4.2023)
urltlm = "https://www.uvek-gis.admin.ch/BFE/kml/20230418_AlpineSolaranlagen/Stromnetz_swissTLM.gpkg"
dfStationTLM = gpd.read_file(urltlm, layer="TLM_NUTZUNGSAREAL")

#ToDo: Daten direkt aus STAC API ziehen

## 3. Swissgrid Unterwerke von den swissTLM3D Unterwerken subtrahieren
Nur die Unterwerke der Swissgrid sind in der Netzebene 2. Aus diesem Grund können die Unterwerke von Swissgrid den Unterwerken im swissTLM3D abgezogen werden. Unterwerke welche übrig bleiben, entsprechen der gesuchten Netzeben 4.

In [ ]:
# Ein SpatialJoin machen und alle Objekte filtern welche keine xtf_id haben. Damit bleiben die UWs der NE4
dfDifference = gpd.sjoin(dfStationenSwissgrid, dfStationTLM ,how='right',predicate='intersects')

# Entspricht den TLM UWs der Netzebene 4 (muss noch weiter bereinigt werden)
dfTLM_NE4 = dfDifference[dfDifference['xtf_id'].isnull()]
# Entspricht den TLM UWs der Netzebene 2 (Swissgrid)
dfTLM_NE2 = dfDifference[dfDifference['xtf_id'].notnull()]



In [ ]:
# Prüfen welche Swissgrid UWs nicht mit den TLM-Daten gejoint wurden und als Dataframe speichern
dfStationenSwissgrid['TlmMapped'] = dfStationenSwissgrid['xtf_id'].isin(dfTLM_NE2['xtf_id'])
dfStationenSwissgrid = dfStationenSwissgrid.loc[dfStationenSwissgrid.TlmMapped==False]

In [ ]:
# Die noch nicht geojointen Swissgrid UWs werden gepuffert und erneut gejoint.
dfStationenSwissgrid['geometry'] = dfStationenSwissgrid['geometry'].buffer(100,resolution=16)
dfTLM_NE4 = dfTLM_NE4.drop(columns=['index_left'],axis=1)

In [ ]:
dfDifference = gpd.sjoin(dfStationenSwissgrid, dfTLM_NE4 ,how='right',predicate='intersects')

dfTLM_NE2 = dfDifference[dfDifference['xtf_id_left'].notnull()]
dfTLM_NE4 = dfDifference[dfDifference['xtf_id_left'].isnull()]

In [ ]:
# Die noch vorhandenen UWs der NE2 werden wieder als Punkte dargestellt damit besser verglichen werden kann.
dfTLM_NE2 = dfTLM_NE2.drop(columns='geometry')
dfTLM_NE2 = gpd.GeoDataFrame(dfTLM_NE2, geometry=gpd.points_from_xy(dfTLM_NE2._x_left,dfTLM_NE2._y_left))
dfTLM_NE2 = dfTLM_NE2.set_crs('epsg:2056')

In [ ]:
#Karte mit allen Unterwerken der Netzebene 4 und den UWs der NE2 welche noch nicht gejoint wurden

newMap = leafmap.Map(height="600px", width="1000px")
#newMap.add_basemap("SwissFederalGeoportal.NationalMapColor")
newMap.add_gdf(dfTLM_NE2, layer_name="Netzebene 4")
newMap.add_gdf(dfTLM_NE4, layer_name="Swissgrid")

#In dieser Karte sind die noch verbliebenen NE2 UWs zu sehen. In der Karte ist zu sehen, dass keine UW-Polygone mehr vorhanden sind. Daher können die Daten der UW4 exportiert werden.
newMap

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

### 4. Gefilterte Daten speichern

In [ ]:
dfTLM_NE4.to_file("Unterwerke_Netzebene4.gpkg", layer='UnterwerkeNE4', diver="GPKG")

In [ ]:
#ToDo
# mit Luftfahrthindernissen abgleichen